In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import multiprocessing
from torch import autograd
from fastai.conv_learner import *
from fastai.transforms import TfmType
from fasterai.transforms import *
from fasterai.images import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.callbacks import *
from fasterai.loss import *
from fasterai.modules import *
from fasterai.training import *
from fasterai.generators import *
from fastai.torch_imports import *
from pathlib import Path
from itertools import repeat
import random
import tensorboardX
torch.cuda.set_device(3)
plt.style.use('dark_background')
torch.backends.cudnn.benchmark=True


from shutil import copyfile
from scipy import misc

In [ ]:
IMAGENET = Path('data/imagenet/ILSVRC/Data/CLS-LOC/test')
RESCLASS = Path('data/resclass')
LOWRES_TRAIN = RESCLASS/'train/lowres'
HIGHRES_TRAIN = RESCLASS/'train/highres'
LOWRES_VALID = RESCLASS/'valid/lowres'
HIGHRES_VALID = RESCLASS/'valid/highres'
proj_id = 'ResCriticPre11'
TENSORBOARD_PATH = Path('data/tensorboard/' + proj_id)

dpath = IMAGENET.parent/(proj_id + '_crit_128.h5')

files =list(IMAGENET.glob("*.*"))
numfiles = len(files)
print(numfiles)
numtrain = int(0.9 * numfiles)
numvalid = numfiles - numtrain
sizes=[64,96,128,160]

train_files = files[:numtrain]
valid_files = files[numtrain:]

lres_train_files = train_files[:numtrain//2]
hres_train_files = train_files[numtrain//2:]

lres_valid_files = train_files[:numvalid//2]
hres_valid_files = train_files[numvalid//2:]

for file in lres_train_files:
    image = open_image(str(file))
    shrunk=scale_min(image,sizes[random.randint(0, len(sizes)-1)])
    upsized=scale_min(shrunk, 192)
    result_path = LOWRES_TRAIN/file.name
    misc.imsave(result_path, np.clip(upsized,0,1))
    
for file in lres_valid_files:
    image = open_image(str(file))
    shrunk=scale_min(image,sizes[random.randint(0, len(sizes)-1)])
    upsized=scale_min(shrunk, 192)
    result_path = LOWRES_VALID/file.name
    misc.imsave(result_path, np.clip(upsized,0,1))
    
for file in hres_train_files:
    image = open_image(str(file))
    resized=scale_min(image, 192)
    result_path = HIGHRES_TRAIN/file.name
    misc.imsave(result_path, np.clip(resized,0,1))
    
    
for file in hres_valid_files:
    image = open_image(str(file))
    resized=scale_min(image, 192)
    result_path = HIGHRES_VALID/file.name
    misc.imsave(result_path, np.clip(resized,0,1))
    
    

In [ ]:
lr=1e-3
lrs = np.array([lr,lr,lr])

sz=128
keep_pct=1.0
bs=32
torch.backends.cudnn.benchmark=True

## Training

In [ ]:
def create_model(pretrained:bool=False):
    return DCCriticDense(ni=3, nf=128)

In [ ]:
arch=resnet34
data = ImageClassifierData.from_paths(RESCLASS, bs=bs, tfms=tfms_from_model(arch, sz))

In [ ]:
wd=1e-7

In [ ]:
learn = ConvLearner.pretrained(create_model, data, pretrained=False)

In [ ]:
#learn.lr_find()

In [ ]:
#learn.sched.plot()

In [ ]:
learn.fit(lrs, 1, cycle_len=1, wds=wd, use_clr_beta=(10,10,0.95,0.85))

In [ ]:
learn.fit(lrs, 1, cycle_len=1, wds=wd, use_clr_beta=(10,10,0.95,0.85))

In [ ]:
#learn.save('superres_crit_pretrain')

In [ ]:
save_model(learn.model, str(dpath))